To get virtual environment installed for ESM, I did the following:

`conda create --name esm python=3.7
sudo apt-get install python-pip
sudo apt update
sudo apt install python3-pip
sudo apt-get install libjpeg-dev
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116
pip install fair-esm
pip install git+https://github.com/facebookresearch/esm.git  # bleeding edge, current repo main branch
git clone https://github.com/facebookresearch/esm.git
conda install -c anaconda ipykernel
python -m ipykernel install --user --name=esm
conda install -c conda-forge tqdm
sudo pip3 install tqdm 
pip install matplotlib pandas seaborn scipy
conda install scikit-learn`

# Import statements

In [5]:
import random
from collections import Counter
from tqdm import tqdm

import torch
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

import esm

import scipy
from sklearn.model_selection import cross_validate
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression, SGDRegressor
from sklearn.pipeline import Pipeline

# Convert benchmark peptide dataset to embeddings

In [ ]:
# cannot handle Js in peptide data! replace with L for all J
peptide = pd.read_csv('./data/JV_data/peptides.csv')
outpath = "./data/JV_data/peptides_for_embedding"
file = open(outpath + '.fasta', 'w')
index = 0
vals = []
seqs = []
for seq, on in zip(list(peptide['seq']), list(peptide['target'])):
    seq = seq.replace("J", "L")
    file.write(">" + str(index) + '|' + str(index) + '|' + str(on) + "\n" + seq + "\n")
    index = index + 1
file.close() #do not forget to close it

To convert sequences to embeddings, run: 

`python scripts/extract.py esm2_t33_650M_UR50D examples/data/JV_data/peptides_for_embedding.fasta examples/data/JV_data/peptides_for_embedding --repr_layers 0 32 33 --include mean per_tok`

# Load embeddings and generate train/test splits for cross-validation

In [7]:
FASTA_PATH = "./data/JV_data/peptides_for_embedding.fasta"
EMB_PATH = "./data/JV_data/peptides_for_embedding"

EMB_LAYER = 33

In [8]:
ys = []
Xs = []
for header, _seq in esm.data.read_fasta(FASTA_PATH):
    scaled_effect = header.split('|')[-1]
    ys.append(float(scaled_effect))
    fn = f'{EMB_PATH}/{header}.pt'
    embs = torch.load(fn)
    Xs.append(embs['mean_representations'][EMB_LAYER])
Xs = torch.stack(Xs, dim=0).numpy()
print(len(ys))
print(Xs.shape)

67769
(67769, 1280)


In [9]:
train_size = 0.8
Xs_train, Xs_test, ys_train, ys_test = train_test_split(Xs, ys, train_size=train_size, random_state=42)

Xs_train.shape, Xs_test.shape, len(ys_train), len(ys_test)

((54215, 1280), (13554, 1280), 54215, 13554)

# Train three types of scikit-learn models and evaluate cross-validation performance

In [14]:
_scoring = ['r2']
_cv = 3

results = cross_validate(estimator=KNeighborsRegressor(),
                               X=Xs,
                               y=ys,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
print(results)

{'fit_time': array([0.20871258, 0.20620513, 0.21264458]), 'score_time': array([38.52070427, 40.65353727, 38.65091777]), 'test_r2': array([0.05416103, 0.15476717, 0.17504163]), 'train_r2': array([0.5480586 , 0.54198079, 0.49187272])}


In [15]:
results = cross_validate(estimator=RandomForestRegressor(),
                               X=Xs,
                               y=ys,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
print(results)

{'fit_time': array([11234.61773443, 11404.13718867, 11250.20415258]), 'score_time': array([1.59572792, 1.58039427, 1.51511288]), 'test_r2': array([0.10818455, 0.12395164, 0.17530255]), 'train_r2': array([0.91410091, 0.91030729, 0.90278354])}


In [16]:
results = cross_validate(estimator=SVR(),
                               X=Xs,
                               y=ys,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
print(results)

{'fit_time': array([1836.60960579, 1839.91163206, 1846.5775888 ]), 'score_time': array([1196.26056409, 1200.58589315, 1137.9080894 ]), 'test_r2': array([0.0246183 , 0.0461565 , 0.05675271]), 'train_r2': array([0.21208438, 0.19719572, 0.12409563])}


# Test external test set performance - compute embeddings

In [ ]:
# cannot handle Js in peptide data! replace with L for all J
peptide = pd.read_csv('./data/JV_data/TESTSET_peptides.csv')
outpath = "./data/JV_data/TESTSET_peptides_for_embedding"
file = open(outpath + '.fasta', 'w')
index = 0
vals = []
seqs = []
for seq, on in zip(list(peptide['seq']), list(peptide['target'])):
    seq = seq.replace("J", "L")
    file.write(">" + str(index) + '|' + str(index) + '|' + str(on) + "\n" + seq + "\n")
    index = index + 1
file.close() #do not forget to close it

As before, run below to convert sequences to embeddings: 
    
`python scripts/extract.py esm2_t33_650M_UR50D examples/data/JV_data/TESTSET_peptides_for_embedding.fasta examples/data/JV_data/TESTSET_peptides_for_embedding --repr_layers 0 32 33 --include mean per_tok`

# Load embeddings for external test set

In [1]:
TESTFASTA_PATH = "./data/JV_data/TESTSET_peptides_for_embedding.fasta"
TESTEMB_PATH = "./data/JV_data/TESTSET_peptides_for_embedding"

In [6]:
ys_test = []
Xs_test = []
for header, _seq in esm.data.read_fasta(TESTFASTA_PATH):
    scaled_effect = header.split('|')[-1]
    ys_test.append(float(scaled_effect))
    fn = f'{TESTEMB_PATH}/{header}.pt'
    embs = torch.load(fn)
    Xs_test.append(embs['mean_representations'][EMB_LAYER])
Xs_test = torch.stack(Xs_test, dim=0).numpy()
print(len(ys_test))
print(Xs_test.shape)

471
(471, 1280)


In [7]:
Xs_train.shape, Xs_test.shape, len(ys_train), len(ys_test)

((67769, 1280), (471, 1280), 67769, 471)

# Test three scikit-learn models on external test sets

In [9]:
knn = KNeighborsRegressor()
knn.fit(Xs_train, ys_train)
y_preds = knn.predict(Xs_test)
print(f'{scipy.stats.spearmanr(ys_test, y_preds)}')
slope, intercept, r_val, p_val, std_error = scipy.stats.linregress(ys_test, y_preds)
print(f'{r_val ** 2}') # r2

SpearmanrResult(correlation=0.664371808241041, pvalue=2.7514495971080336e-61)
0.4107362125400169


In [10]:
rfr = RandomForestRegressor()
rfr.fit(Xs_train, ys_train)
y_preds = rfr.predict(Xs_test)
print(f'{scipy.stats.spearmanr(ys_test, y_preds)}')
slope, intercept, r_val, p_val, std_error = scipy.stats.linregress(ys_test, y_preds)
print(f'{r_val ** 2}') # r2

SpearmanrResult(correlation=0.8340329946971428, pvalue=3.225625228286665e-123)
0.7304124943507356


In [11]:
svr = SVR()
svr.fit(Xs_train, ys_train)
y_preds = svr.predict(Xs_test)
print(f'{scipy.stats.spearmanr(ys_test, y_preds)}')
slope, intercept, r_val, p_val, std_error = scipy.stats.linregress(ys_test, y_preds)
print(f'{r_val ** 2}') # r2

SpearmanrResult(correlation=0.6259867349874955, pvalue=1.3094123401043502e-52)
0.4910201495353209
